# Chains in LangChain

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [2]:
import pandas as pd
df = pd.read_csv('Data.csv')
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


### LLMChain

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [4]:
llm = ChatOpenAI(temperature=0.9)

In [5]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that nakes {product}?"
)

In [6]:
chain = LLMChain(llm=llm, prompt=prompt)

In [7]:
product = "Full Stack web applications with the lastest technologies and powered by AI"
chain.run(product)

'TechFusion+ or AIStackTech'

### SimpleSequentialChain

In [8]:
from langchain.chains import SimpleSequentialChain

In [9]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
        a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [10]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company: {company_name}"
)

# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [11]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                             )

In [12]:
overall_simple_chain(product)



> Entering new SimpleSequentialChain chain...
TechStack AI Solutions
TechStack AI Solutions is a cutting-edge technology company that specializes in developing AI solutions for businesses to streamline and automate processes.

> Finished chain.


{'input': 'Full Stack web applications with the lastest technologies and powered by AI',
 'output': 'TechStack AI Solutions is a cutting-edge technology company that specializes in developing AI solutions for businesses to streamline and automate processes.'}

### SequentialChain

In [13]:
from langchain.chains import SequentialChain

In [15]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to English:"
    "\n\n{Review}"
)
# chain 1: input= Review, output= English review
chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key='English_Review'
                     )

In [16]:
second_prompt = ChatPromptTemplate.from_template(
    "Summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review, output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="summary"
                     )

In [17]:
# prompt template 3: Translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)

# chain 3: input= Review, output= Language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                       )

In [19]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the follwing"
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)

# chain 4: input: summary, language, output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                      )

In [20]:
# Overall_chain: input= Review, output= English_Review, summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary", "followup_message"],
    verbose=True
)

In [21]:
review=df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam does not hold, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer expresses dissatisfaction with the taste and foam of the product, suspecting that it might be an old batch or counterfeit compared to the same ones purchased in stores.',
 'followup_message': "Réponse de suivi: Nous sommes désolés d'apprendre que vous n'êtes pas satisfait du goût et de la mousse de notre produit. Nous tenons à vous assurer que nos produits sont authentiques et nous prenons la qualité très au sérieux. Il est possible qu'il s'agisse d'un lot ancien ou contrefait, ce qui n'est pas conforme à nos normes. Nous vous prions de nous excuser pour cette expérience et nous vous invitons à nous contacter dir

### Router Chain

In [22]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

therapist_template = """ You are a very successful therapist and psychologist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [23]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "therapist", 
        "description": "Good for answering therapist and psychology questions", 
        "prompt_template": therapist_template
    }
]

In [24]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

In [25]:
llm = ChatOpenAI(temperature=0)

In [26]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [27]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [28]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [29]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=['input'],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [30]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                         )

In [31]:
chain.run("I am 26 years old, I have a bakground in electronics engineering but I haven't start my professional life. I feel a bit depressed since I think it is getting late and I am getting old. I might loose my opportunity to become profesional and have a stable life")



> Entering new MultiPromptChain chain...


c:\Users\juanb\anaconda3\lib\site-packages\langchain\chains\llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


therapist: {'input': "I am 26 years old, I have a background in electronics engineering but I haven't started my professional life. I feel a bit depressed since I think it is getting late and I am getting old. I might lose my opportunity to become professional and have a stable life."}
> Finished chain.


"I understand that you may be feeling anxious and overwhelmed about your current situation. It's important to remember that age is just a number and it's never too late to pursue your goals and dreams. Here are a few steps you can take to address your concerns:\n\n1. Self-reflection: Take some time to reflect on your interests, passions, and long-term goals. Consider what you truly enjoy and what career path aligns with your skills and interests. This will help you gain clarity and direction.\n\n2. Research and explore: Once you have identified your interests, research the various career options available in your field of electronics engineering. Look for opportunities that match your skills and qualifications. Explore different industries and companies that may offer stable and fulfilling careers.\n\n3. Networking: Connect with professionals in your field through networking events, online platforms, or industry conferences. Seek advice and guidance from experienced individuals who can